In [ ]:
import tkinter as tk
import random

class Joueur:
    def __init__(self, identifiant, strategie, total_tours=10):
        self.id = identifiant
        self.strategie = strategie
        self.pieces = 5
        self.derniere_action = True  # Pour les stratégies qui en ont besoin
        self.pertes_consecutives = 0  # Pour le pacifiste
        self.tour_actuel = 0  # Pour les tricheurs conditionnels
        self.total_tours = total_tours  # Pour MiParcours

    def choisir_action(self, dernier_choix_adversaire=None):
        self.tour_actuel += 1
        if self.strategie == "gentil":
            return True
        elif self.strategie == "méchant":
            return False
        elif self.strategie == "aléatoire":
            return random.choice([True, False])
        elif self.strategie == "copieur" and dernier_choix_adversaire is not None:
            return dernier_choix_adversaire
        elif self.strategie == "opportuniste":
            return self.derniere_action  # Continue de coopérer sauf s'il se fait avoir
        elif self.strategie == "pacifiste":
            return self.pertes_consecutives < 2  # Coopère sauf s'il a perdu deux fois d'affilée
        elif self.strategie == "vengeur" and dernier_choix_adversaire is not None:
            return dernier_choix_adversaire  # Triche si l'autre a trahi au dernier tour
        elif self.strategie == "tricheur_pair":
            return self.tour_actuel % 2 != 0  # Coopère aux tours impairs, triche aux tours pairs
        elif self.strategie == "tricheur_impair":
            return self.tour_actuel % 2 == 0  # Coopère aux tours pairs, triche aux tours impairs
        elif self.strategie == "mi_parcours":
            return self.tour_actuel <= self.total_tours // 2  # Coopère au début, triche après la moitié des tours
        return True

class Jeu:
    def __init__(self, root):
        self.joueurs = [
            Joueur(1, "gentil"), Joueur(2, "méchant"), 
            Joueur(3, "aléatoire"), Joueur(4, "copieur"),
            Joueur(5, "opportuniste"), Joueur(6, "pacifiste"), Joueur(7, "vengeur"),
            Joueur(8, "tricheur_pair"), Joueur(9, "tricheur_impair"), Joueur(10, "mi_parcours")
        ]
        self.root = root
        self.label = tk.Label(root, text=self.afficher_etat())
        self.label.pack()
        self.bouton_tour = tk.Button(root, text="Jouer un tour", command=self.jouer_tour)
        self.bouton_tour.pack()
    
    def jouer_tour(self):
        for i, joueur1 in enumerate(self.joueurs):
            for j, joueur2 in enumerate(self.joueurs):
                if i < j:  # Chaque paire unique
                    choix1 = joueur1.choisir_action(joueur2.derniere_action)
                    choix2 = joueur2.choisir_action(joueur1.derniere_action)
                    
                    if choix1 and choix2:
                        joueur1.pieces += 2
                        joueur2.pieces += 2
                    elif choix1 and not choix2:
                        joueur1.pieces -= 1
                        joueur2.pieces += 3
                        joueur1.pertes_consecutives += 1
                        joueur2.pertes_consecutives = 0
                    elif not choix1 and choix2:
                        joueur1.pieces += 3
                        joueur2.pieces -= 1
                        joueur2.pertes_consecutives += 1
                        joueur1.pertes_consecutives = 0
                    else:
                        joueur1.pertes_consecutives = 0
                        joueur2.pertes_consecutives = 0
                    
                    joueur1.derniere_action = choix1
                    joueur2.derniere_action = choix2
        
        self.joueurs = [j for j in self.joueurs if j.pieces > 0]
        self.label.config(text=self.afficher_etat())
    
    def afficher_etat(self):
        return "\n".join([f"Joueur {j.id} ({j.strategie}) - Pièces: {j.pieces}" for j in self.joueurs])

root = tk.Tk()
root.title("Simulation ARE")
jeu = Jeu(root)
root.mainloop()
